In [222]:
# output bench time
!find MGH-100P/*.csv -mmin -60 -exec ls -gohd {} \;
lfeats=!find MGH-100P/*.csv
lfeats=list(lfeats)
print(lfeats)
print(len(lfeats))

['MGH-100P/features_Case1_seg12.mat.csv', 'MGH-100P/new_features_Case101_seg1.mat.csv', 'MGH-100P/new_features_Case101_seg2.mat.csv', 'MGH-100P/new_features_Case101_seg4.mat.csv', 'MGH-100P/new_features_Case101_seg6.mat.csv', 'MGH-100P/new_features_Case102_seg1.mat.csv', 'MGH-100P/new_features_Case102_seg3.mat.csv', 'MGH-100P/new_features_Case102_seg4.mat.csv', 'MGH-100P/new_features_Case102_seg6.mat.csv', 'MGH-100P/new_features_Case103_seg6.mat.csv', 'MGH-100P/new_features_Case103_seg9.mat.csv', 'MGH-100P/new_features_Case104_seg2.mat.csv', 'MGH-100P/new_features_Case104_seg4.mat.csv', 'MGH-100P/new_features_Case105_seg1.mat.csv', 'MGH-100P/new_features_Case105_seg2.mat.csv', 'MGH-100P/new_features_Case105_seg4.mat.csv', 'MGH-100P/new_features_Case105_seg5.mat.csv', 'MGH-100P/new_features_Case106_seg1.mat.csv', 'MGH-100P/new_features_Case106_seg3.mat.csv', 'MGH-100P/new_features_Case107_seg10.mat.csv', 'MGH-100P/new_features_Case107_seg1.mat.csv', 'MGH-100P/new_features_Case107_seg2.m

In [179]:
#!ls -l Case5
#!ls -l pid
#lsub=!find Case*/*.mat -size -100000
lsub=!find Case*/*.mat -type f -size +300M -size -600M -exec ls {}\;

print(len(lsub))
print(lsub)
#!ls -l Case102/Case102_seg3.mat

1
["find: missing argument to `-exec'"]


In [93]:
sc.addPyFile("fcn_shannon_entro.py")
sc.addPyFile("full_pipeline.py")
sc.addPyFile("compute_spectrogram_sunhaoqi.py")
sc.addPyFile("get_features.py")
sc.addPyFile("runtime.py")
sc.addPyFile("mtspec.py")

In [167]:
lfiles=!ls Case10*/*.mat

#lfiles=list(lfiles)
lsub=lfiles
print(lsub)
print(len(lsub))

['Case101/Case101_seg1.mat', 'Case101/Case101_seg2.mat', 'Case101/Case101_seg3.mat', 'Case101/Case101_seg4.mat', 'Case101/Case101_seg5.mat', 'Case101/Case101_seg6.mat', 'Case102/Case102_seg1.mat', 'Case102/Case102_seg2.mat', 'Case102/Case102_seg3.mat', 'Case102/Case102_seg4.mat', 'Case102/Case102_seg5.mat', 'Case102/Case102_seg6.mat', 'Case103/Case103_seg1.mat', 'Case103/Case103_seg2.mat', 'Case103/Case103_seg3.mat', 'Case103/Case103_seg4.mat', 'Case103/Case103_seg5.mat', 'Case103/Case103_seg6.mat', 'Case103/Case103_seg8.mat', 'Case103/Case103_seg9.mat', 'Case104/Case104_seg1.mat', 'Case104/Case104_seg2.mat', 'Case104/Case104_seg3.mat', 'Case104/Case104_seg4.mat', 'Case105/Case105_seg1.mat', 'Case105/Case105_seg2.mat', 'Case105/Case105_seg3.mat', 'Case105/Case105_seg4.mat', 'Case105/Case105_seg5.mat', 'Case105/Case105_seg6.mat', 'Case105/Case105_seg7.mat', 'Case106/Case106_seg1.mat', 'Case106/Case106_seg2.mat', 'Case106/Case106_seg3.mat', 'Case107/Case107_seg10.mat', 'Case107/Case107_s

In [ ]:
import itertools
import os
import numpy as np
import hdf5storage
import spectrum
from joblib import Parallel
from joblib import delayed
from numpy.core.defchararray import lower
from numpy import floor
import swiftclient
import pandas as pd
#from compute_spectrogram_sunhaoqi import mtspecgram_shq
from get_features import get_features
from scipy.signal import butter, filtfilt, lfilter
from runtime import cellarray, matlabarray, cat, cell, arange, size, disp, find, dot, copy, length, Dict2Obj,catstr
import scipy

from mtspec import mt_spectrogram

def filtering_data(temp,Fs):
    fnyq=Fs / 2  # nyquist frequency
    fc_high=50
    fc_low=0.5        
    b, a = butter(6, [fc_low / fnyq, fc_high / fnyq], 'bandpass')
    out=filtfilt(b,a,temp,padtype='odd',padlen=3*(np.amax((len(a), len(b))) - 1))
    return out


def mockFunc(elt):
    try:
        #d=dict(sc._conf.getAll())
        #print(d['spark.executor.id'])
        
        absdir='/gpfs/fs01/user/seff-34c2f0d3dcc620-a916a00b641d/notebook/work/'
        elt=absdir+elt
        dataset=os.path.getsize(elt)
        matfile = hdf5storage.loadmat(elt)
        
      
        Fs = cellarray(int(np.round(matfile['Fs'])))
        start_time = cellarray(matfile['start_time'])
        channels = cellarray(matfile['channels'])
        data = cellarray(matfile['data'])
    

        ## New: Resample to 200 Hz!!! %
        #Fs =int( Fs.item())
        #P = 200
        #Q = Fs
        #print('resample',Q,P)
        #data = scipy.signal.resample(data.T, int(len(data.T) * P / Q))
        ### data = np.resample(data, 200, Fs)
        #data = cellarray(data.T)
        Fs = 200

        def strrep_s(a, x, y):
            return a.replace(x, y)

        # =strcmp(labels,"['f8']")
        strrep = np.vectorize(strrep_s)

        print('channels', channels)
        print(size(channels, 1))
        # df_s0=pd.DataFrame()
        labels = copy(channels)
        # print(labels.shape)

        for i in arange(1, size(channels, 1)).reshape(-1):
            x = strrep(channels[i, :], ' ', '')
            labels[i] = lower(x)

        disp('load 0')
        disp('compute 1','calculate locational differential voltage')
        # pd=copy(pwd)
        movingwin = matlabarray(cat(2, 2))

        # dictobj
        params = Dict2Obj({'default': 1})

        params.pad = copy(0)
        params.fpass = copy(cat(0.5, 20))
        params.err = copy(0)
        params.trialave = copy(0)
        params.tapers = copy(cat(2, 3))
        params.Fs = copy(200)

        # spect=cell(4,4)
        # COI=numpy.empty((4, ), dtype=object)
        COI = cell(4, )
        COI[1] = cellarray([['fp1', 'f7'], ['f7', 't3'], ['t3', 't5'], ['t5', 'o1']])
        COI[2] = cellarray([['fp1', 'f3'], ['f3', 'c3'], ['c3', 'p3'], ['p3', 'o1']])
        COI[3] = cellarray([['fp2', 'f4'], ['f4', 'c4'], ['c4', 'p4'], ['p4', 'o2']])
        COI[4] = cellarray([['fp2', 'f8'], ['f8', 't4'], ['t4', 't6'], ['t6', 'o2']])

        # test
        def strcmp_s(x, y):
            return x == y

        strcmp = np.vectorize(strcmp_s)

        dataB = cell(4, 4)

        # print(data.shape,data.__class__,dataB.shape,dataB.__class__,data.dtype,dataB.dtype)
        for kk in arange(1, 4).reshape(-1):
            # print('kk',kk)
            coi = COI[kk]
            print('coi', coi, size(coi, 1), size(coi, 2), size(coi))

            for k in arange(1, size(coi, 1)).reshape(-1):
                print('kk', kk, 'k', k, coi[k, 1], coi[k, 2])
                # print(coi[k, 1 - 1], coi[k, 2 - 1])
                # print('k',k,coi,'labels')
                # print(coi[k,1-1],coi[k,2-1])

                c1 = find(strcmp(labels, coi[k, 1]))
                c2 = find(strcmp(labels, coi[k, 2]))
                c1 = c1.item()
                c2 = c2.item()
                # dataB[kk,k]=data(c1,arange()) - data(c2,arange())
                # print('items',c1,c2)
                dataB[kk, k] = np.array((data[c1, :] - data[c2, :]).tolist())    
    

        # Par version
        n_jobs = -1  # number of cpus for parallel computing, -1 is all cpus
        verbose = True  # verbosity in parallel computing
        res = Parallel(n_jobs=n_jobs, verbose=verbose)(
            delayed(mt_spectrogram)(dataB[kk,k], Fs) for kk,k in itertools.product([1,2,3,4],[
                1,2,3,4])
        )

        
        # Step 3
        spect = cell(4, 4)
        print(len(res))

        for i, rr in enumerate(res):
            # rr[0][freq_good_ids]
            ii=i % 4
            iii=floor(i/4)
            spect[iii, ii + 1] = rr[0].T

        stimes=res[0][1]
        sfreqs = res[0][2]
        
        
 
        R = cell(4, 1)
        # R=np.empty((4,1),dtype=object)
        for kk in arange(1, 4).reshape(-1):
            # print('kk',kk,'size',size(spect[kk,1-1]),spect[kk,1-1].shape,tuple(size(spect[kk,1-1])))
            T = np.zeros(size(spect[kk, 1]))
            # print('T',T)
            # T = zeros(sizespect[kk, 1 - 1].shape[0])
            print('T', T.shape)
            for k in arange(1, 4).reshape(-1):
                # print('kk',kk,'k',k)

                T = T + spect[kk, k]
            R[kk] = T / 4



        epoch_length = dot(Fs, 2)
        window_length = dot(cat(14, 10, 6, 2), Fs)

        # filtering of data
        fnyq = Fs / 2  # nyquist frequency
        fc_high = 50
        fc_low = 0.5
        #  cutoff frequencies
        # b,a=butter(6,cat(fc_low / fnyq,fc_high / fnyq),'bandpass',nargout=2)
        b, a = butter(6, cat(fc_low / fnyq, fc_high / fnyq), 'bandpass')

        for kk in arange(1, 4).reshape(-1):
            for k in arange(1, 4).reshape(-1):
                # print('kk',kk,'k',k)
                temp = dataB[kk, k]
                # dataB[kk, k] = filtfilt(b, a, temp)
                # dataB[kk,k]=filtfilt(b,a,temp,padtype='odd')

                # dataB[kk, k] = lfilter(b, a, temp)
                dataB[kk, k] = filtfilt(b, a, temp, padtype='odd', padlen=3 * (np.amax((len(a), len(b))) - 1))
                
        
        # Step 5
        spectwindow_length = floor(window_length / Fs / 2)

        N = length(dataB[1, 1])
        nr_epochs = floor((N - (window_length[1] - epoch_length) - Fs) / epoch_length)
        nr_features = 144

        features = cell(4, nr_features, nr_epochs)
        for kk in arange(1, 4).reshape(-1):
            
            spect = R[kk, 1]
            data = dataB[kk, :]

            #Par version
            n_jobs = -1 # number of cpus for parallel computing, -1 is all cpus
            verbose = True  # verbosity in parallel computing
            res = Parallel(n_jobs=n_jobs, verbose=verbose)(
                # delayed(compute_spec_each_seg)(eeg[window_start[wi]:window_start[wi] + window_length, :], NW, Fs) for wi in range(window_num)
                delayed(get_features)(data, int(i.item()), window_length, spect, sfreqs, spectwindow_length, Fs) for i
                in arange(1, nr_epochs).reshape(-1)
            )

            print(len(res))
            for i, rr in enumerate(res):
                # rr[0][freq_good_ids]
                features[kk, :, i + 1] = rr[:]

        features = features.reshape(cat(dot(4, nr_features), length(features)))
        features = features.T            
            
            
        # Step 6
        fileName0=elt
        saveFeats = features.T

        # turn this into a pandas df
        saveFeats = pd.DataFrame(saveFeats)

        # add time stamp
        saveFeats.reset_index(inplace=True)
        fileNum = fileName0[:len(fileName0) - 4]
        import re
        fileNum = re.search('(\d+)$', fileNum).group(0)
        sampleRate = round(Fs)

        def timeStamp(ind, fileNum, sampleRate):
            start = (int(fileNum) - 1) * 99999
            row = start + int(ind)
            time = row * int(sampleRate)
            return time

        saveFeats['time'] = saveFeats['index'].apply(lambda e: timeStamp(e, fileNum, sampleRate))

        # add case id
        caseNum = int(re.search(r'\d+', fileName0).group())
        saveFeats['case'] = caseNum

        try:
            print(fileName0)
            onlyFiName = os.path.basename(fileName0)
            fp = os.path.basename(fileName0) + '.csv'
            print(fp)
            savep = absdir + 'MGH-100P/new_features_' + fp
            print('save to ', savep)
            if not os.path.isfile(savep):
                saveFeats.to_csv(savep)
        
        except:
            print('[ERROR]', 'saving to csv')
        
        
        
        #Fs=matfile['Fs'].item()
        #dataset=matfile['data']
        #start_time=matfile['start_time']

        #Parallelize spectrogram
        #n_jobs=-1
        #verbose=10
        #res = Parallel(n_jobs=n_jobs, verbose=verbose)(
        #   delayed(mt_spectrogram)(cellarray(matfile['data'][i,:]), Fs) for i in range(1,16))
        msg='No issues'
    except Exception as e:
        print('error')
        #msg=str(e)
        #print('error')
        Fs=''
        start_time=''
        dataset=(0,0)
        dataB=(0,0)
        msg=str(e)        
        #dataset=dataset.shape
        
        
        ## Parallelize filtering
        #out = Parallel(n_jobs=n_jobs, verbose=verbose)(
        #   delayed(filtering_data)(data[i],Fs) for i in range(16))

    #print(dataB.shape)
    return (elt, start_time, Fs, dataset, dataB,msg)

    
import os
import hdf5storage
import numpy as np
import multiprocessing as mp

def testFunc(elt):
    try:
        absdir='/gpfs/fs01/user/seff-34c2f0d3dcc620-a916a00b641d/notebook/work/'
        elt=absdir+elt
        dataset=os.path.getsize(elt)
        matfile = hdf5storage.loadmat(elt)
        
        cpu_count=mp.cpu_count()
        
      
        Fs = int(np.round(matfile['Fs']))
        start_time = matfile['start_time']
        channels = matfile['channels']
        data = matfile['data']
        msg='No issue'
    except Exception as e:
        print('error')
        msg=str(e)
        
    return (elt,start_time,data,data.shape,cpu_count,msg)

# Benching 8
runt=True
if (runt):
    #!ls MGH-100P
    
    #print('num exec',sc._jsc.sc().getExecutorMemoryStatus().size())
    n_exec=sc._jsc.sc().getExecutorMemoryStatus().size()
    
    #distData = sc.parallelize(lsub[:n_exec])
    distData = sc.parallelize(lsub)
    dtd=distData.repartition(min(n_exec,len(lsub)))
    distData.cache()
    dtd.cache()
    
    print('num part',dtd.getNumPartitions())
    print('num data',len(lsub))
    print('num exec',sc._jsc.sc().getExecutorMemoryStatus().size())
    print('data',lsub)
    
    testdtd=dtd.map(lambda e: mockFunc(e))
    testdtd.cache()
    %time lres=testdtd.take(len(lsub))    
    print(dtd.collect(),lres,len(lres))

num part 59
num data 76
num exec 59
data ['Case101/Case101_seg1.mat', 'Case101/Case101_seg2.mat', 'Case101/Case101_seg3.mat', 'Case101/Case101_seg4.mat', 'Case101/Case101_seg5.mat', 'Case101/Case101_seg6.mat', 'Case102/Case102_seg1.mat', 'Case102/Case102_seg2.mat', 'Case102/Case102_seg3.mat', 'Case102/Case102_seg4.mat', 'Case102/Case102_seg5.mat', 'Case102/Case102_seg6.mat', 'Case103/Case103_seg1.mat', 'Case103/Case103_seg2.mat', 'Case103/Case103_seg3.mat', 'Case103/Case103_seg4.mat', 'Case103/Case103_seg5.mat', 'Case103/Case103_seg6.mat', 'Case103/Case103_seg8.mat', 'Case103/Case103_seg9.mat', 'Case104/Case104_seg1.mat', 'Case104/Case104_seg2.mat', 'Case104/Case104_seg3.mat', 'Case104/Case104_seg4.mat', 'Case105/Case105_seg1.mat', 'Case105/Case105_seg2.mat', 'Case105/Case105_seg3.mat', 'Case105/Case105_seg4.mat', 'Case105/Case105_seg5.mat', 'Case105/Case105_seg6.mat', 'Case105/Case105_seg7.mat', 'Case106/Case106_seg1.mat', 'Case106/Case106_seg2.mat', 'Case106/Case106_seg3.mat', 'Case1

In [10]:
# Segment Load
import os
import numpy as np
import hdf5storage
import spectrum
from joblib import Parallel
from joblib import delayed
from numpy.core.defchararray import lower
from numpy import floor
import swiftclient
import pandas as pd
#from compute_spectrogram_sunhaoqi import mtspecgram_shq
from get_features import get_features
from scipy.signal import butter, filtfilt, lfilter
from runtime import cellarray, matlabarray, cat, cell, arange, size, disp, find, dot, copy, length, Dict2Obj,catstr

from mtspec import mt_spectrogram

def load(fileName0):
    #matfile = hdf5storage.loadmat(fp)
    
    #return matfile

    absdir='/gpfs/fs01/user/seff-34c2f0d3dcc620-a916a00b641d/notebook/work/'

    fileName0=absdir+fileName0
    print(fileName0)
    
    #if not os.path.isfile(fileName0):
    #    #print('writing to local',fileName0)    
    #    container = 'MGH'
    #    resp_headers, obj_contents = conn.get_object(container, fileName0)
    #    #with open(os.path.basename(fileName0), 'wb') as local:
    #    with open(fileName0, 'wb') as local:
    #        local.write(obj_contents)

    #print('load file',fileName0)
    matfile = hdf5storage.loadmat(fileName0)
    #matfile = hdf5storage.loadmat(os.path.basename(fileName0))

    Fs=cellarray(int(np.round(matfile['Fs'])))
    start_time = cellarray(matfile['start_time'])
    channels=cellarray(matfile['channels'])
    data = cellarray(matfile['data'])
    

    # New: Resample to 200 Hz!!! %    
    Fs=Fs.item()
    print('Fs',Fs)
    # data=scipy.signal.resample(data.T,200,200)
    P=200
    Q=int(Fs)
    data=scipy.signal.resample(data.T, int(len(data.T) * P / Q))
    ## data = np.resample(data, 200, Fs)
    data = cellarray(data.T)
    Fs = 200


    def strrep_s(a,x,y):
        return a.replace(x,y)

    #=strcmp(labels,"['f8']")
    strrep=np.vectorize(strrep_s)

    print('channels',channels)
    print(size(channels,1))
    # df_s0=pd.DataFrame()
    labels=copy(channels)
    print(labels.shape)
    
    for i in arange(1, size(channels, 1)).reshape(-1):
        x=strrep(channels[i,:],' ','')
        labels[i] = lower(x)

    disp('load 0')
    disp('compute 1')
    # pd=copy(pwd)
    movingwin=matlabarray(cat(2,2))

    # dictobj
    params=Dict2Obj({'default':1})

    params.pad = copy(0)
    params.fpass = copy(cat(0.5,20))
    params.err = copy(0)
    params.trialave = copy(0)
    params.tapers = copy(cat(2,3))
    params.Fs = copy(200)

    # spect=cell(4,4)
    # COI=numpy.empty((4, ), dtype=object)
    COI=cell(4,)
    COI[1]=cellarray([['fp1','f7'],['f7','t3'],['t3','t5'],['t5','o1']])
    COI[2]=cellarray([['fp1','f3'],['f3','c3'],['c3','p3'],['p3','o1']])
    COI[3]=cellarray([['fp2','f4'],['f4','c4'],['c4','p4'],['p4','o2']])
    COI[4]=cellarray([['fp2','f8'],['f8','t4'],['t4','t6'],['t6','o2']])

    print('COI',COI)
    
    ROInickname=cellarray(['LL','LP','RP','RL'])
    print(ROInickname)

    # test
    def strcmp_s(x,y):
        return x == y

    
    strcmp=np.vectorize(strcmp_s)
    #=strcmp(labels,"['f8']")
    # print(strcmp(ROInickname,'LL'))

    dataB=cell(4,4)
    
    # print(data.shape,data.__class__,dataB.shape,dataB.__class__,data.dtype,dataB.dtype)
    for kk in arange(1 , 4 ).reshape(-1):
        # print('kk',kk)
        coi=COI[kk]
        print('coi',coi,size(coi,1),size(coi,2),size(coi))

        for k in arange(1 , size(coi, 1) ).reshape(-1):
            print('kk',kk,'k',k , coi[k,1],coi[k,2])
            # print(coi[k, 1 - 1], coi[k, 2 - 1])
            # print('k',k,coi,'labels')
            # print(coi[k,1-1],coi[k,2-1])

            c1=find(strcmp(labels,coi[k,1]))
            c2=find(strcmp(labels,coi[k,2]))
            c1=c1.item()
            c2 = c2.item()
            #dataB[kk,k]=data(c1,arange()) - data(c2,arange())
            print('items',c1,c2)
            dataB[kk, k] = np.array((data[c1, :] - data[c2, :]).tolist())

    #d={}
    #d['dataB']=dataB
    return dataB


In [ ]:
from itertools import product
import glob
# Create a (case, channel, window) rdd
#def optim_load_spectrogram(fileName0):
def loadData_seg(lst):
    for e in lst:
        param=e.split('_')
    #hdf5storage
    
    return l

def optim_load_spectrogram(dir0):    
    #mat=fileName0
    
    #fn=os.path.basename(fileName0)
    
    
    
    win_size=100
    absdir='/gpfs/fs01/user/seff-34c2f0d3dcc620-a916a00b641d/notebook/work'
    rdd0=[dir0+'/'+os.path.basename(elt) for elt in glob.glob(absdir+'/'+dir0+'/*.mat')]
    
    rdd01=[]
    for elt in rdd0:
        #print(elt)
        dataB=load(elt)
        #dataB='elt'
    
        rdd1=[str(k1)+'_'+str(k2) for k1,k2 in product( [1,2,3,4], [1,2,3,4] )]
        
        rdd01.append([(str(e1)+'_'+str(e2),dataB[e2[0],e2[1]]) for e1,e2 in product(rdd0,rdd1)])
        #rdd01=[(str(e1)+'_'+str(e2),dataB[e2[0],e2[1]]) for e1,e2 in product(rdd0,rdd1)]
        
        #rdd01.append([(str(e1)+'_'+str(e2)) for e1,e2 in product(rdd0,rdd1)])
    
    #winsize
    #rdd012=[elt+'_'+str(i) for elt,i in product(rdd01,range(1,win_size))]
    
    #rdddata=loadData_seg(rdd01)
    
    # load_data
    return rdd01

#tsData=['Case6/Case6_seg1.mat','Case6/Case6_seg2.mat']
tsData=['Case5','Case6','Case4']
rdd=sc.parallelize(tsData)
rdd.repartition(len(tsData))
print(rdd.getNumPartitions())

l=rdd.flatMap(lambda e: optim_load_spectrogram(e))

%time combined=l.collect()
print(len(combined))
print(combined)


2


In [79]:
# FEATS
ln=!ls -l --time-style=+"|%Y-%m-%d_%H:%M:%S|" MGH-100P/*.csv | awk '{print $5,$6,$7}'
l=[e.split(' ') for e in ln]
df_feats=pd.DataFrame(l,columns=['feat_size','feat_create_time','feat_filename'])
df_feats['token']=df_feats.feat_filename.str.extract('.*(Case.*)\.csv',expand=True)


#print(l)
# MAT
#ln=!ls -l Case*/*.mat | awk '{print $5,$9}' | grep 'mat'
##print(ln)
#l=[e.split(' ') for e in ln]
#df_mats=pd.DataFrame(l,columns=['size','filename'])
## scale to pyspark
#df_mats['token']=df_mats.filename.str.extract('.*(Case.*)',expand=True)

In [138]:
# Test broadcast
#Fs=200
#sc.broadcast(Fs)
import h5py
import numpy as np
chr_s=np.vectorize(lambda e: chr(e))

def load_h5(elt):
#elt='Case5/Case5_seg1.mat'
    absdir='/gpfs/fs01/user/seff-34c2f0d3dcc620-a916a00b641d/notebook/work/'
    elt=absdir+elt
    
    os.system('touch /gpfs/fs01/user/seff-34c2f0d3dcc620-a916a00b641d/notebook/work/pid/%s ' % elt)
    try:
        with h5py.File(elt, 'r') as fp:
            l=fp['start_time'][:]
            #l=dataset
            l=[chr(e) for e in l]
            s=''.join(l)
            start_time=s
            print(start_time)

            l=fp['Fs'][:]
            Fs=l.item()
            print(Fs)

            l=fp['channels'][:].T
            l=chr_s(l)
            channels=[''.join(elt) for elt in l]
            #print(l)

            #l=fp['data'][:]
            #data=l.T
            #print(data.shape)
    except:
        start_time,data,channels,Fs='','','',''
        
    return (start_time,data,channels,Fs)




In [140]:
ln=!ls -l Case*/*.mat | awk '{print $5,$9}' | grep 'mat'
print(ln)
l=[e.split(' ') for e in ln]
df_mats=pd.DataFrame(l,columns=['size','filename'])
# scale to pyspark
df_mats['token']=df_mats.filename.str.extract('.*(Case.*)',expand=True)

df_mats=df_mats[0:10]
print(df_mats.head())
print(len(df_mats))

from pyspark.sql import SQLContext
from pyspark.sql import functions as F


print('num exec',sc._jsc.sc().getExecutorMemoryStatus().size())
n_exec=sc._jsc.sc().getExecutorMemoryStatus().size()

l_data=list(df_mats.ix[:10,'filename'])
sqlctx=SQLContext(sc)
rdd=sc.parallelize(l)

# Dataframe
#rdd=rdd.map(lambda e: e.split(' '))
#s=rdd.collect()
#sdf=sqlctx.createDataFrame(s).show()
print(l_data)
resRdd=rdd.repartition(10)
resRdd.cache()
dataRdd=resRdd.map(lambda e: load_h5(e))
data=dataRdd.collect()

print(data)

#sqlCtx = SQLContext(sc)
#sqlCtx.createDataFrame(df).show()

#print(sdf)


['578104054 Case86/Case86_seg1.mat', '1096842818 Case86/Case86_seg2.mat', '1161036307 Case86/Case86_seg3.mat', '656064928 Case86/Case86_seg4.mat', '1114130078 Case87/Case87_seg1.mat', '1134503145 Case87/Case87_seg2.mat', '860641912 Case87/Case87_seg3.mat', '1124481622 Case87/Case87_seg4.mat', '1151680241 Case88/Case88_seg1.mat', '331460354 Case88/Case88_seg2.mat', '155046257 Case88/Case88_seg3.mat', '944810229 Case88/Case88_seg4.mat', '1088126215 Case88/Case88_seg5.mat', '449743871 Case88/Case88_seg6.mat', '69792334 Case88/Case88_seg7.mat', '972731098 Case89/Case89_seg1.mat', '673094788 Case89/Case89_seg2.mat', '31401940 Case89/Case89_seg3.mat', '711770823 Case89/Case89_seg4.mat', '735714479 Case89/Case89_seg5.mat', '661416598 Case89/Case89_seg6.mat', '584447357 Case89/Case89_seg7.mat', '1080331034 Case8/Case8_seg1.mat', '759021302 Case8/Case8_seg2.mat', '1112372751 Case8/Case8_seg3.mat', '912344960 Case8/Case8_seg4.mat', '260749860 Case8/Case8_seg5.mat', '35786830 Case8/Case8_seg6.mat

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 18.0 failed 10 times, most recent failure: Lost task 1.9 in stage 18.0 (TID 374, yp-spark-dal09-env5-0019, executor c4ee6b02-8d4e-4049-b0da-bff8c2793302): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/src/spark21master/spark-2.1.0-bin-2.7.3/python/lib/pyspark.zip/pyspark/worker.py", line 174, in main
    process()
  File "/usr/local/src/spark21master/spark-2.1.0-bin-2.7.3/python/lib/pyspark.zip/pyspark/worker.py", line 169, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/src/spark21master/spark-2.1.0-bin-2.7.3/python/lib/pyspark.zip/pyspark/serializers.py", line 268, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "<ipython-input-140-c014b8d4ad0d>", line 30, in <lambda>
  File "<

In [81]:
import pandas as pd
from pandasql import sqldf
# pandasql is sqlite syntax

ln=!ls -l --time-style=+"|%Y-%m-%d_%H:%M:%S|" Case*/*.mat | awk '{print $5,$6,$7}'

#l=ln.split('\n')
#print(ln)
l=[e.split(' ') for e in ln]
dfl=pd.DataFrame(l,columns=['src_size','src_create_time','src_filename'])
dfl['token']=dfl.src_filename.str.extract('.*/(Case.*)',expand=True)
dfl.head()


dfout=sqldf("select * from (select *, cast(i0.src_size as decimal)/cast(r0.feat_size as decimal) as ratio from dfl i0 left join df_feats r0 on i0.token=r0.token) where ratio > 0",globals())
print(dfout)
#from pyspark import SQLContext
#sqlctx=SQLContext(sc)
#sdf = sqlctx.createDataFrame(dfout)
#print(sdf.printSchema())
##sqldf("select * from dfl i0 order by cast(size as integer) left join df_feats r0 on i0.token=r0.token")

       src_size        src_create_time               src_filename  \
0      41984256  |2017-07-04_22:11:17|   Case101/Case101_seg1.mat   
1     854189583  |2017-07-04_22:11:36|   Case101/Case101_seg2.mat   
2     885963922  |2017-07-04_22:13:04|   Case101/Case101_seg6.mat   
3     868371141  |2017-07-04_22:13:20|   Case102/Case102_seg1.mat   
4    1223459230  |2017-07-04_22:14:11|   Case102/Case102_seg4.mat   
5      38349277  |2017-07-04_22:14:24|   Case102/Case102_seg6.mat   
6      26548279  |2017-07-04_22:15:50|   Case103/Case103_seg6.mat   
7      41487984  |2017-07-04_23:02:53|   Case103/Case103_seg9.mat   
8    1565238074  |2017-07-04_23:03:43|   Case104/Case104_seg2.mat   
9    1339184685  |2017-07-04_23:05:15|   Case105/Case105_seg2.mat   
10     28870262  |2017-07-04_23:05:38|   Case105/Case105_seg5.mat   
11    780845365  |2017-07-04_23:07:05|   Case106/Case106_seg3.mat   
12     44725884  |2017-07-04_23:07:08|   Case107/Case107_seg1.mat   
13    943603415  |2017-07-04_23:07

In [77]:
from __future__ import print_function
#[elt for elt in sc._conf.getAll()]

d=dict(sc._conf.getAll())
print(d['spark.executor.id'])

#sc.getConf.getAll.foreach(print)
print(sc._jsc.sc().getExecutorMemoryStatus().size())

driver
1


In [65]:
# windows speed up

!mkdir MGH-100P-SPECTRO

In [ ]:
# %load get_features.py

import hdf5storage
from matplotlib.mlab import prctile
from numpy import dot
from numpy import mean, std, ceil
from scipy.stats import kurtosis

from fcn_shannon_entro import fcn_shannon_entro

# get_features.m
import numpy as np
import scipy


def bandpower(Pxx, f, frange, opt='psd'):
    """ integrate the power spectral density between fmin and fmax
        using the trapezoidal method
    """
    (fmin, fmax) = frange
    ind_min = scipy.argmax(f > fmin) - 1
    ind_max = scipy.argmax(f > fmax) + 1
    # print(ind_min,ind_max)
    return Pxx[ind_min: ind_max].sum(axis=0) * np.diff(f)[0]


from runtime import cellarray, matlabarray, cat, length, arange, zeros, disp, floor, cell, end


def get_features(data=None, i=None, window_length=None, spect=None, sfreqs=None, spectwindow_length=None, Fs=None,
                 *args, **kwargs):
    # time features: line length, kurtosis, shannon entropy and nonlinear energy.
    # spectral features: delta, theta, alpha and beta bandpower. delta to theta
    # ratio, theta to alpha ratio and delta to alpha ratio. Kurtosis of delta,
    # theta, alpha and beta bandpower.
    if (i%100) == 0:
        print("compute", i,'fast')

    line1avg = 0
    line2avg = 0
    line3avg = 0
    line4avg = 0

    kurt1avg = 0
    kurt2avg = 0
    kurt3avg = 0
    kurt4avg = 0

    shan1avg = 0
    shan2avg = 0
    shan3avg = 0
    shan4avg = 0

    psy1meanavg = 0
    psy2meanavg = 0
    psy3meanavg = 0
    psy4meanavg = 0

    psy1stdavg = 0
    psy2stdavg = 0
    psy3stdavg = 0
    psy4stdavg = 0


    # window_length = [14 10 6 2]*Fs; # the different windows for each segment
    t = dot((i - 1), window_length[4]) + dot(0.5, window_length[1])
    for k in arange(1, 4).reshape(-1):
        # win1=data[k][arange(t - dot(0.5,window_length[1]) + 1,t + dot(0.5,window_length[1]))]
        # win2=data[k][arange(t - dot(0.5,window_length[2]) + 1,t + dot(0.5,window_length[2]))]
        # win3=data[k][arange(t - dot(0.5,window_length[3]) + 1,t + dot(0.5,window_length[3]))]
        # win4=data[k][arange(t - dot(0.5,window_length[4]) + 1,t + dot(0.5,window_length[4]))]
        
        win1o = data[k][t - dot(0.5, window_length[1]):t + dot(0.5, window_length[1])]
        win2o = data[k][t - dot(0.5, window_length[2]):t + dot(0.5, window_length[2])]
        win3o = data[k][t - dot(0.5, window_length[3]):t + dot(0.5, window_length[3])]
        win4o = data[k][t - dot(0.5, window_length[4]):t + dot(0.5, window_length[4])]

        win1 = matlabarray(data[k][t - dot(0.5, window_length[1]):t + dot(0.5, window_length[1])])
        win2 = matlabarray(data[k][t - dot(0.5, window_length[2]):t + dot(0.5, window_length[2])])
        win3 = matlabarray(data[k][t - dot(0.5, window_length[3]):t + dot(0.5, window_length[3])])
        win4 = matlabarray(data[k][t - dot(0.5, window_length[4]):t + dot(0.5, window_length[4])])

        # time features
        #for j in arange(1, window_length[1] - 1).reshape(-1):
            # line1 = line1 + abs(win1[J + 1] - win1[j])
        J=arange(1, window_length[1] - 1).reshape(-1)
        line1 = np.sum(np.abs(win1[J + 1] - win1[J])) / (window_length[1] - 1)

        # for j in arange(1, window_length[2] - 1).reshape(-1):
        #     line2 = line2 + abs(win2[j + 1] - win2[j])
        J=arange(1, window_length[2] - 1).reshape(-1)
        line2 = np.sum(np.abs(win2[J + 1] - win2[J])) / (window_length[2] - 1)

        # for j in arange(1, window_length[3] - 1).reshape(-1):
        #     line3 = line3 + abs(win3[j + 1] - win3[j])
        J=arange(1, window_length[3] - 1).reshape(-1)
        line3 = np.sum(np.abs(win3[J + 1] - win3[J])) / (window_length[3] - 1)

        # for j in arange(1, window_length[4] - 1).reshape(-1):
        #     line4 = line4 + abs(win4[j + 1] - win4[j])
        J=arange(1, window_length[4] - 1).reshape(-1)
        line4 = np.sum(np.abs(win4[J + 1] - win4[J])) / (window_length[4] - 1)

        kurt1 = kurtosis(win1o, 0)
        kurt2 = kurtosis(win2o, 0)
        kurt3 = kurtosis(win3o, 0)
        kurt4 = kurtosis(win4o, 0)

        #     samp_en1 = fcn_SampEn(2, 0.2*std(win1), win1); # sample entropy
        #     samp_en2 = fcn_SampEn(2, 0.2*std(win2), win2);
        #     samp_en3 = fcn_SampEn(2, 0.2*std(win3), win3);
        #     samp_en4 = fcn_SampEn(2, 0.2*std(win4), win4);
        shan1 = abs(fcn_shannon_entro(win1.T))
        shan2 = abs(fcn_shannon_entro(win2.T))
        shan3 = abs(fcn_shannon_entro(win3.T))
        shan4 = abs(fcn_shannon_entro(win4.T))


        psy1 = matlabarray(np.zeros((1, length(win1) - 3)))
        # for n in arange(4, length(win1)).reshape(-1):
        #     psy1[n - 3] = dot(win1[n - 1], win1[n - 2]) - dot(win1[n], win1[n - 3])

        N=arange(4, length(win1)).reshape(-1)
        # psy1[1,N - 3] = dot(win1[N - 1], win1[N - 2]) - dot(win1[N], win1[N - 3])
        psy1[1, N - 3] = win1[N - 1] * win1[N - 2] - win1[N] * win1[N - 3]

        psy1mean = mean(abs(psy1))
        psy1std = std(psy1, ddof=1)


        psy2 = matlabarray(np.zeros((1, length(win2) - 3)))

        # for n in arange(4, length(win2)).reshape(-1):
        #     psy2[n - 3] = dot(win2[n - 1], win2[n - 2]) - dot(win2[n], win2[n - 3])
        N=arange(4, length(win2)).reshape(-1)
        # psy2[N - 3] = dot(win2[N - 1], win2[N - 2]) - dot(win2[N], win2[N - 3])
        psy2[1, N - 3] = win2[N - 1] * win2[N - 2] - win2[N] * win2[N - 3]

        psy2mean = mean(abs(psy2))
        psy2std = std(psy2, ddof=1)


        psy3 = matlabarray(np.zeros((1, length(win3) - 3)))

        # for n in arange(4, length(win3)).reshape(-1):
        #     psy3[n - 3] = dot(win3[n - 1], win3[n - 2]) - dot(win3[n], win3[n - 3])
        N=arange(4, length(win3)).reshape(-1)
        # psy3[N - 3] = dot(win3[N - 1], win3[N - 2]) - dot(win3[N], win3[N - 3])
        psy3[1, N - 3] = win3[N - 1] * win3[N - 2] - win3[N] * win3[N - 3]

        psy3mean = mean(abs(psy3))
        psy3std = std(psy3, ddof=1)


        psy4 = matlabarray(np.zeros((1, length(win4) - 3)))

        # for n in arange(4, length(win4)).reshape(-1):
        #     psy4[n - 3] = dot(win4[n - 1], win4[n - 2]) - dot(win4[n], win4[n - 3])
        N=arange(4, length(win4)).reshape(-1)
        # psy4[N - 3] = dot(win4[N - 1], win4[N - 2]) - dot(win4[N], win4[N - 3])
        psy4[1, N - 3] = win4[N - 1] * win4[N - 2] - win4[N] * win4[N - 3]

        psy4mean = mean(abs(psy4))
        psy4std = std(psy4, ddof=1)

        line1avg = line1avg + line1
        line2avg = line2avg + line2
        line3avg = line3avg + line3
        line4avg = line4avg + line4

        kurt1avg = kurt1avg + kurt1
        kurt2avg = kurt2avg + kurt2
        kurt3avg = kurt3avg + kurt3
        kurt4avg = kurt4avg + kurt4

        shan1avg = shan1avg + shan1
        shan2avg = shan2avg + shan2
        shan3avg = shan3avg + shan3
        shan4avg = shan4avg + shan4

        psy1meanavg = psy1meanavg + psy1mean
        psy2meanavg = psy2meanavg + psy2mean
        psy3meanavg = psy3meanavg + psy3mean
        psy4meanavg = psy4meanavg + psy4mean

        psy1stdavg = psy1stdavg + psy1std
        psy2stdavg = psy2stdavg + psy2std
        psy3stdavg = psy3stdavg + psy3std
        psy4stdavg = psy4stdavg + psy4std

    line1avg = line1avg / 4
    line2avg = line2avg / 4
    line3avg = line3avg / 4
    line4avg = line4avg / 4

    kurt1avg = kurt1avg / 4
    kurt2avg = kurt2avg / 4
    kurt3avg = kurt3avg / 4
    kurt4avg = kurt4avg / 4

    shan1avg = shan1avg / 4
    shan2avg = shan2avg / 4
    shan3avg = shan3avg / 4
    shan4avg = shan4avg / 4

    psy1meanavg = psy1meanavg / 4
    psy2meanavg = psy2meanavg / 4
    psy3meanavg = psy3meanavg / 4
    psy4meanavg = psy4meanavg / 4

    psy1stdavg = psy1stdavg / 4
    psy2stdavg = psy2stdavg / 4
    psy3stdavg = psy3stdavg / 4
    psy4stdavg = psy4stdavg / 4

    # spectral features
    #### t1 = ceil(t/Fs/2);
    tt1 = ceil(t / Fs / 2 - dot(0.5, spectwindow_length))
    tt2 = ceil(t / Fs / 2 + dot(0.5, spectwindow_length))
    tt1 = tt1.astype(int)
    tt2 = tt2.astype(int)

    # edit: matrix select---
    # spectwin1=spect[:,tt1[1] + 1:tt2[1]]
    # spectwin2=spect[:,tt1[2] + 1:tt2[2]]
    # spectwin3=spect[:,tt1[3] + 1:tt2[3]]
    # spectwin4=spect[:,tt1[4] + 1:tt2[4]]
    spectwin1 = spect[:, tt1[1]:tt2[1]]
    spectwin2 = spect[:, tt1[2]:tt2[2]]
    spectwin3 = spect[:, tt1[3]:tt2[3]]
    spectwin4 = spect[:, tt1[4]:tt2[4]]

    eps = 0.0
    delta1 = bandpower(spectwin1, sfreqs, cat(1, 4), 'psd')
    theta1 = bandpower(spectwin1, sfreqs, cat(4, 8), 'psd')
    alpha1 = bandpower(spectwin1, sfreqs, cat(8, 12), 'psd')
    beta1 = bandpower(spectwin1, sfreqs, cat(12, 18), 'psd')
    total1 = bandpower(spectwin1, sfreqs, cat(1, sfreqs[len(sfreqs) - 2]), 'psd') + eps
    # total1 = bandpower(spectwin1, sfreqs, cat(1, sfreqs[end()]), 'psd')

    delta1_rat = delta1 / total1
    theta1_rat = theta1 / total1
    alpha1_rat = alpha1 / total1
    beta1_rat = beta1 / total1

    delta_theta1 = delta1 / (theta1 + eps)
    delta_alpha1 = delta1 / (alpha1 + eps)
    theta_alpha1 = theta1 / (alpha1 + eps)

    kurtdelta1 = kurtosis(delta1, 0)
    kurttheta1 = kurtosis(theta1, 0)
    kurtalpha1 = kurtosis(alpha1, 0)
    kurtbeta1 = kurtosis(beta1, 0)

    delta1_mean = mean(delta1_rat)
    theta1_mean = mean(theta1_rat)
    alpha1_mean = mean(alpha1_rat)
    beta1_mean = mean(beta1_rat)

    delta1_min = np.min(delta1_rat)
    theta1_min = np.min(theta1_rat)
    alpha1_min = np.min(alpha1_rat)
    beta1_min = np.min(beta1_rat)

    delta1_std = std(delta1_rat)
    theta1_std = std(theta1_rat)
    alpha1_std = std(alpha1_rat)
    beta1_std = std(beta1_rat)

    delta1_prct = prctile(delta1_rat, 95)
    theta1_prct = prctile(theta1_rat, 95)
    alpha1_prct = prctile(alpha1_rat, 95)
    beta1_prct = prctile(beta1_rat, 95)

    delthe1_mean = mean(delta_theta1)
    delalph1_mean = mean(delta_alpha1)
    thealph1_mean = mean(theta_alpha1)

    delthe1_min = min(delta_theta1)
    delalph1_min = min(delta_alpha1)
    thealph1_min = min(theta_alpha1)

    delthe1_std = std(delta_theta1)
    delalph1_std = std(delta_alpha1)
    thealph1_std = std(theta_alpha1)

    delthe1_prct = prctile(delta_theta1, 95)
    delalph1_prct = prctile(delta_alpha1, 95)
    thealph1_prct = prctile(theta_alpha1, 95)

    delta2 = bandpower(spectwin2, sfreqs, cat(sfreqs[1], 4), 'psd')
    theta2 = bandpower(spectwin2, sfreqs, cat(4, 8), 'psd')
    alpha2 = bandpower(spectwin2, sfreqs, cat(8, 12), 'psd')
    beta2 = bandpower(spectwin2, sfreqs, cat(12, 18), 'psd')
    total2 = bandpower(spectwin2, sfreqs, cat(1, sfreqs[len(sfreqs) - 2]), 'psd') + eps
    # total2=bandpower(spectwin2,sfreqs,'psd') + eps
    # total2=beta2

    delta2_rat = delta2 / total2
    theta2_rat = theta2 / total2
    alpha2_rat = alpha2 / total2
    beta2_rat = beta2 / total2

    delta_theta2 = delta2 / (theta2 + eps)
    delta_alpha2 = delta2 / (alpha2 + eps)
    theta_alpha2 = theta2 / (alpha2 + eps)

    kurtdelta2 = kurtosis(delta2, 0)
    kurttheta2 = kurtosis(theta2, 0)
    kurtalpha2 = kurtosis(alpha2, 0)
    kurtbeta2 = kurtosis(beta2, 0)

    delta2_mean = mean(delta2_rat)
    theta2_mean = mean(theta2_rat)
    alpha2_mean = mean(alpha2_rat)
    beta2_mean = mean(beta2_rat)

    delta2_min = np.min(delta2_rat)
    theta2_min = np.min(theta2_rat)
    alpha2_min = np.min(alpha2_rat)
    beta2_min = np.min(beta2_rat)

    delta2_std = std(delta2_rat)
    theta2_std = std(theta2_rat)
    alpha2_std = std(alpha2_rat)
    beta2_std = std(beta2_rat)

    delta2_prct = prctile(delta2_rat, 95)
    theta2_prct = prctile(theta2_rat, 95)
    alpha2_prct = prctile(alpha2_rat, 95)
    beta2_prct = prctile(beta2_rat, 95)

    delthe2_mean = mean(delta_theta2)
    delalph2_mean = mean(delta_alpha2)
    thealph2_mean = mean(theta_alpha2)

    delthe2_min = np.min(delta_theta2)
    delalph2_min = np.min(delta_alpha2)
    thealph2_min = np.min(theta_alpha2)

    delthe2_std = std(delta_theta2)
    delalph2_std = std(delta_alpha2)
    thealph2_std = std(theta_alpha2)

    delthe2_prct = prctile(delta_theta2, 95)
    delalph2_prct = prctile(delta_alpha2, 95)
    thealph2_prct = prctile(theta_alpha2, 95)

    delta3 = bandpower(spectwin3, sfreqs, cat(sfreqs[1], 4), 'psd')
    theta3 = bandpower(spectwin3, sfreqs, cat(4, 8), 'psd')
    alpha3 = bandpower(spectwin3, sfreqs, cat(8, 12), 'psd')
    beta3 = bandpower(spectwin3, sfreqs, cat(12, 18), 'psd')
    # total3=bandpower(spectwin3,sfreqs,'psd') + eps
    total3 = bandpower(spectwin3, sfreqs, cat(1, sfreqs[len(sfreqs) - 2]), 'psd') + eps

    delta3_rat = delta3 / total3
    theta3_rat = theta3 / total3
    alpha3_rat = alpha3 / total3
    beta3_rat = beta3 / total3

    delta_theta3 = delta3 / (theta3 + eps)
    delta_alpha3 = delta3 / (alpha3 + eps)
    theta_alpha3 = theta3 / (alpha3 + eps)

    kurtdelta3 = kurtosis(delta3, 0)
    kurttheta3 = kurtosis(theta3, 0)
    kurtalpha3 = kurtosis(alpha3, 0)
    kurtbeta3 = kurtosis(beta3, 0)

    delta3_mean = mean(delta3_rat)
    theta3_mean = mean(theta3_rat)
    alpha3_mean = mean(alpha3_rat)
    beta3_mean = mean(beta3_rat)

    delta3_min = np.min(delta3_rat)
    theta3_min = np.min(theta3_rat)
    alpha3_min = np.min(alpha3_rat)
    beta3_min = np.min(beta3_rat)

    delta3_std = std(delta3_rat)
    theta3_std = std(theta3_rat)
    alpha3_std = std(alpha3_rat)
    beta3_std = std(beta3_rat)

    delta3_prct = prctile(delta3_rat, 95)
    theta3_prct = prctile(theta3_rat, 95)
    alpha3_prct = prctile(alpha3_rat, 95)
    beta3_prct = prctile(beta3_rat, 95)

    delthe3_mean = mean(delta_theta3)
    delalph3_mean = mean(delta_alpha3)
    thealph3_mean = mean(theta_alpha3)

    delthe3_min = np.min(delta_theta3)
    delalph3_min = np.min(delta_alpha3)
    thealph3_min = np.min(theta_alpha3)

    delthe3_std = std(delta_theta3)
    delalph3_std = std(delta_alpha3)
    thealph3_std = std(theta_alpha3)

    delthe3_prct = prctile(delta_theta3, 95)
    delalph3_prct = prctile(delta_alpha3, 95)
    thealph3_prct = prctile(theta_alpha3, 95)

    delta4 = bandpower(spectwin4, sfreqs, cat(sfreqs[1], 4), 'psd')
    theta4 = bandpower(spectwin4, sfreqs, cat(4, 8), 'psd')
    alpha4 = bandpower(spectwin4, sfreqs, cat(8, 12), 'psd')
    beta4 = bandpower(spectwin4, sfreqs, cat(12, 18), 'psd')
    # total4=bandpower(spectwin4,sfreqs,'psd') + eps
    total4 = bandpower(spectwin4, sfreqs, cat(1, sfreqs[len(sfreqs) - 2]), 'psd') + eps

    delta4_rat = delta4 / total4
    theta4_rat = theta4 / total4
    alpha4_rat = alpha4 / total4
    beta4_rat = beta4 / total4

    delta_theta4 = delta4 / (theta4 + eps)
    delta_alpha4 = delta4 / (alpha4 + eps)
    theta_alpha4 = theta4 / (alpha4 + eps)

    # kurtdelta4 = kurtosis(delta4);
    # kurttheta4 = kurtosis(theta4);
    # kurtalpha4 = kurtosis(alpha4);
    # kurtbeta4 = kurtosis(beta4);
    delta4_mean = mean(delta4_rat)
    theta4_mean = mean(theta4_rat)
    alpha4_mean = mean(alpha4_rat)
    beta4_mean = mean(beta4_rat)

    delta4_min = np.min(delta4_rat)
    theta4_min = np.min(theta4_rat)
    alpha4_min = np.min(alpha4_rat)
    beta4_min = np.min(beta4_rat)

    delta4_std = std(delta4_rat)
    theta4_std = std(theta4_rat)
    alpha4_std = std(alpha4_rat)
    beta4_std = std(beta4_rat)

    delta4_prct = prctile(delta4_rat, 95)
    theta4_prct = prctile(theta4_rat, 95)
    alpha4_prct = prctile(alpha4_rat, 95)
    beta4_prct = prctile(beta4_rat, 95)

    delthe4_mean = mean(delta_theta4)
    delalph4_mean = mean(delta_alpha4)
    thealph4_mean = mean(theta_alpha4)

    delthe4_min = np.min(delta_theta4)
    delalph4_min = np.min(delta_alpha4)
    thealph4_min = np.min(theta_alpha4)

    delthe4_std = std(delta_theta4)
    delalph4_std = std(delta_alpha4)
    thealph4_std = std(theta_alpha4)

    delthe4_prct = prctile(delta_theta4, 95)
    delalph4_prct = prctile(delta_alpha4, 95)
    thealph4_prct = prctile(theta_alpha4, 95)

    line1avg = line1avg.item()
    line2avg = line2avg.item()
    line3avg = line3avg.item()
    line4avg = line4avg.item()


    features1 = matlabarray(
        cat([line1avg], [line2avg], [line3avg], [line4avg], [kurt1avg], [kurt2avg], [kurt3avg], [kurt4avg]))
    features1a = matlabarray(
        cat([shan1avg], [shan2avg], [shan3avg], [shan4avg], [psy1meanavg], [psy2meanavg], [psy3meanavg], [psy4meanavg],
            [psy1stdavg], [psy2stdavg], [psy3stdavg], [psy4stdavg]))
    features1aa = matlabarray(
        cat([delta1_mean], [delta2_mean], [delta3_mean], [delta4_mean], [theta1_mean], [theta2_mean], [theta3_mean],
            [theta4_mean], [alpha1_mean], [alpha2_mean], [alpha3_mean], [alpha4_mean]))
    features1b = matlabarray(
        cat([beta1_mean], [beta2_mean], [beta3_mean], [beta4_mean], [delta1_min], [delta2_min], [delta3_min],
            [theta1_min], [theta2_min], [theta3_min], [alpha1_min], [alpha2_min], [alpha3_min], [beta1_min],
            [beta2_min], [beta3_min], [delta1_std], [delta2_std], [delta3_std], [theta1_std], [theta2_std],
            [theta3_std], [alpha1_std], [alpha2_std], [alpha3_std], [beta1_std], [beta2_std], [beta3_std],
            [delta1_prct], [delta2_prct], [delta3_prct], [theta1_prct], [theta2_prct], [theta3_prct], [alpha1_prct],
            [alpha2_prct], [alpha3_prct], [beta1_prct], [beta2_prct], [beta3_prct], [delthe1_mean], [delthe2_mean],
            [delthe3_mean], [delthe4_mean], [delalph1_mean], [delalph2_mean], [delalph3_mean], [delalph4_mean],
            [thealph1_mean], [thealph2_mean], [thealph3_mean], [thealph4_mean], [delthe1_min], [delthe2_min],
            [delthe3_min], [delalph1_min], [delalph2_min], [delalph3_min], [thealph1_min], [thealph2_min],
            [thealph3_min], [delthe1_std], [delthe2_std], [delthe3_std], [delalph1_std], [delalph2_std], [delalph3_std],
            [thealph1_std], [thealph2_std], [thealph3_std], [delthe1_prct], [delthe2_prct], [delthe3_prct],
            [delalph1_prct], [delalph2_prct], [delalph3_prct], [thealph1_prct], [thealph2_prct], [thealph3_prct],
            [kurtdelta1], [kurtdelta2], [kurtdelta3], [kurttheta1], [kurttheta2], [kurttheta3], [kurtalpha1],
            [kurtalpha2], [kurtalpha3], [kurtbeta1], [kurtbeta2], [kurtbeta3]))
    features2 = matlabarray(
        cat([delta4_min], [theta4_min], [alpha4_min], [beta4_min], [delta4_std], [theta4_std], [alpha4_std],
            [beta4_std], [delta4_prct], [theta4_prct], [alpha4_prct], [beta4_prct], [delthe4_min], [delalph4_min],
            [thealph4_min], [delthe4_std], [delalph4_std], [thealph4_std], [delthe4_prct], [delalph4_prct],
            [thealph4_prct]))

    # kurtbeta4];
    features = matlabarray(cat(features1, features1a, features1aa, features1b, features2))
    # print('shape of features',features.shape)
    # features=matlabarray(cat([features1],[features2]))

    return features


class load_mat():
    def __init__(self, filename, mode='r'):
        self.filename = filename
        self.mode = mode

    def __enter__(self):
        print('open')
        # self.open_file = open(self.filename, self.mode)
        return 0

    def __exit__(self, *args):
        # self.open_file.close()
        print('close')

def load(fp):
    matfile=hdf5storage.loadmat(fp)
    return matfile



if __name__ == '__main__':
    pass

